In [2]:
import torch
import torch.nn as nn
from torch import functional as F

In [3]:
f = open('phg.txt', 'r', encoding='UTF-8')
text = f.read()
print(len(text))

1255699


In [4]:
print(text[:1000])

PREFACE: ON SCIENTIFIC 
COGNITION 


1. It is customary to preface a work with an explanation of 
the author’s aim, why he wrote the book, and the relationship 
in which he believes it to stand to other earlier or contemporary 
treatises on the same subject. In the case of a philosophical 
work, however, such an explanation seems not only superfluous 
but, in view of the nature of the subject-matter, even inappro¬ 
priate and misleading. For whatever might appropriately be 
said about philosophy in a preface—say a historical statement 
of the main drift and the point of view, the general content and 
results, a string of random assertions and assurances about 
truth—none of this can be accepted as the way in which to 
expound philosophical truth. Also, since philosophy moves 
essentially in the element of universality, which includes within 
itself the particular, it might seem that here more than in any 
of the other sciences the subject-matter itself, and even in its 
complete nature

In [5]:
block_size = 8 #length of context before character prediction
batch_size = 4 #number of examples simultaneously processed

#make the vocabulary
vocab = sorted(list(set(text)))

#make the encoding and decoding (text to number, number to text)
encoder = {ch:i for i, ch in enumerate(vocab)}
decoder = {i:ch for i, ch in enumerate(vocab)}
def encode_string(string): 
    return [encoder[char] for char in string]
def decode_string(string):
    return ''.join([decoder[char] for char in string])

#split into training and validation sets
data = torch.tensor(encode_string(text), dtype=torch.long)
portion_train = int(0.9*len(data))
training_set=data[:portion_train]
val_set = data[portion_train:]


#make up the batches (including padding + start & end tokens)
def get_batch(type):
    data = training_set if type == 'train' else val_set
    indices = torch.randint(len(data)-block_size, (batch_size, ))
    inputs = torch.stack([data[index: index+block_size] for index in indices], dim=0)
    targets = torch.stack([data[index+1: index+block_size+1] for index in indices], dim=0)
    return inputs, targets

print(get_batch('train'))

(tensor([[79, 25,  1, 75, 74,  1, 80, 68],
        [75, 74,  1, 69, 74,  1, 80, 68],
        [72,  1, 76, 78, 69, 74, 63, 69],
        [34, 49, 37, 38, 32, 30, 41,  1]]), tensor([[25,  1, 75, 74,  1, 80, 68, 65],
        [74,  1, 69, 74,  1, 80, 68, 65],
        [ 1, 76, 78, 69, 74, 63, 69, 76],
        [49, 37, 38, 32, 30, 41,  1, 44]]))


In [13]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(32, head_size, bias=False)
        self.query = nn.Linear(32, head_size, bias=False)
        self.value = nn.Linear(32, head_size, bias=False)
        self.register_buffer('triangle', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        batch, block, channels = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        sa_weights = k @ q.transpose(1, 2) * channels ** -0.5 
        sa_weights = sa_weights.masked_fill(self.triangle[:block, :block]==0, float('-inf'))
        #note to self - why do we do softmax here?
        sa_weights = F.softmax(weights, dim = 2)
        #self-attention weights
#         out = sa_weights @ v
        return sa_weights
    
x = torch.randn(4, 8, 32)
head = Head(16)
print(head(x))

AttributeError: module 'torch.functional' has no attribute 'softmax'